<a href="https://colab.research.google.com/github/Ayana2020/EffectValidation/blob/master/Propensity_analysis_sample_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

傾向スコアマッチングの実装例1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


import statsmodels.api as sm


from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_auc_score

import itertools
import time

NameError: ignored

# データ前処理

In [ ]:
data = pd.read_csv("q_data_x.csv")

In [ ]:
a

In [ ]:
# 共変量
X = data[['TVwatch_day', 'age', 'sex', 'marry_dummy', 'child_dummy', 'inc', 'pmoney','area_kanto', 'area_tokai', 'area_keihanshin', 
          'job_dummy1', 'job_dummy2', 'job_dummy3', 'job_dummy4', 'job_dummy5', 'job_dummy6', 'job_dummy7',
          'fam_str_dummy1', 'fam_str_dummy2', 'fam_str_dummy3', 'fam_str_dummy4']]

# 群別変数
Z = data['cm_dummy']

# 傾向スコア推定モデルの構築

In [ ]:
exog = sm.add_constant(X) # 切片の追加
logit_model = sm.Logit(endog=Z, exog=exog) # ロジスティック回帰
logit_res = logit_model.fit()

# 傾向スコア推定モデルの精度確認

In [ ]:


# AUCの算出

ps = logit_res.predict(exog)


print('AUC = {:.3f}'.format(roc_auc_score(y_true=Z, y_score=ps)))
print('-----ps------\n',ps)

In [ ]:
#キャリブレーションプロットを描画

_, ax1 = plt.subplots(figsize=(10, 5))


prob_true, prob_pred = calibration_curve(y_true=Z, y_prob=ps, n_bins=20)
ax1.plot(prob_pred, prob_true, marker='o', label='calibration plot')
ax1.plot([0,1], [0,1], linestyle='--', color='black', label='ideal')
ax1.legend(bbox_to_anchor=(1.2, 0.9), loc='upper left', borderaxespad=0)
ax1.set_xlabel('predicted probability')
ax1.set_ylabel('true probability')

ax2 = ax1.twinx()
ax2.hist(ps, bins=20, histtype='step', rwidth=0.9)
ax2.set_ylabel('count')
plt.tight_layout()
plt.show()

# ATTの算出

In [ ]:
# ATTの推定
"""ATT= TGにおける介入の効果"""

# 結果変数（アプリ見たかどうか）
Y = data['gamedummy']

# TGのレコード数
sample_size = len(data.loc[data['gamedummy']==1])
print(f"sample_size:{sample_size}")


ATT_list = []

# 1000回試行して、ATTの値の平均 ± x のかたちで算出する
for i in range(1000):

    # 群別変数Z=1のレコードの、結果変数の列の、インデックス（行番号）を取得し、ランダムにTGのデータ数分を抽出して並び替え
    idx1 = pd.Series(data.loc[data['cm_dummy']==1, 'gamedummy'].index).sample(n=sample_size, replace=True, random_state=i)
    
    # CGもTGと同じデータ数分の行番号をランダムに取得する
    idx0 = pd.Series(data.loc[data['cm_dummy']==0, 'gamedummy'].index).sample(n=sample_size, replace=True, random_state=i)
    
    
    # Z（CM見たかどうか）のデータフレームを結合する
    # TGとCGから抽出したランダムな行を結合する
    Z_tmp = np.r_[Z[idx1],Z[idx0]]

    # Y(共変量)のデータフレームを結合する
    Y_tmp = np.r_[Y[idx1],Y[idx0]]
    
    # ps=推定値のデータフレームを結合
    ps_tmp = np.r_[ps[idx1], ps[idx0]]
    
    
    """
    """
    w01_tmp = (1-Z_tmp)*ps_tmp / (1-ps_tmp)
    
    
    """
    ATT = E(Y1 | z=1) - E(Y0  | z=1)
   
　   E(Y1 | z=1) = 実績値
  　 E(Y0 | z=0) = 
    
   """
    
    E1 = np.mean(Y_tmp[Z_tmp==1])
    
    # ?????????????
    E0 = np.sum(Y_tmp * w01_tmp) / np.sum(w01_tmp)
    
    
    ATT = E1 - E0
    
    ATT_list.append(ATT)

In [ ]:
# ATTの

att_avg = np.mean(ATT_list)
att_std = np.std(ATT_list)
att_min = att_avg - att_std * 1.96
att_max = att_avg + att_std * 1.96

print(f'att_avg: {att_avg}')
print(f'att_std: {att_std}')
print(f'att_min: {att_min}')
print(f'att_max: {att_max}')


print('ATT = {:.3f} ± {:.3f} (s.d.={:.3f})'.format(np.mean(ATT_list), np.std(ATT_list)*1.96, np.std(ATT_list)))

# 共変量の分布の確認

In [ ]:

def standardized_mean_difference(X1, X0):
    """標準化平均差(SMD）を算出する"""

    N1 = len(X1)
    N0 = len(X0)
    s_pool = ((N1-1)*np.var(X1)+(N0-1)*np.var(X0))/(N1+N0-2)

    return (np.mean(X1)-np.mean(X0))/np.sqrt(s_pool)


def smd_on_the_treated(X, Z, ps): 
    """傾向スコアを用いた調整前後のSMDを計算する関数"""
    
    
    X1 = X[Z==1]
    X0 = X[Z==0]
    
    ps0 = ps[Z==0]
    
    #_??????
    X10 = X0*ps0/(1-ps0)
    
    
    # マッチング調整前のSMD：TGとCGの間のSMD
    smd_before = standardized_mean_difference(X1, X0)
    
    # マッチング調整後のSMD: TGと、CGのうちTGとマッチングしたデータ群の間のSMD
    smd_after = standardized_mean_difference(X1, X10)


    return smd_before, smd_after

In [ ]:
smd_list = []



# 1共変量ごとに、その共変量のデータセット・Zのデータセット・傾向スコアのデータセットの3つのデータでSMDを算出して格納する
for col_name in X.columns:
    smd_before, smd_after = smd_on_the_treated(X=X[col_name], Z=Z, ps=ps)
    smd_list.append([col_name, smd_before, smd_after])


# smdリストをDFに変換
smd_df = pd.DataFrame(data=smd_list, columns=['covariate', 'SMD(before)', 'SMD(after)'])
smd_df

In [ ]:
# 傾向スコア調整前後の共変量の分布を描画


plt.figure(figsize=(5,10))

# データ
plt.scatter(smd_df['SMD(before)'], smd_df['covariate'], label='SMD(before)')
plt.scatter(smd_df['SMD(after)'], smd_df['covariate'], label='SMD(after)')

# X=０に縦線を引く
# X.shape ➡　Xの行数(10000)、列数(21)
plt.vlines(x=[0], ymin=-1, ymax=X.shape[1])


# グリッドを引く
plt.grid()


plt.xlabel('SMD')
plt.ylabel('covariate')


plt.legend()